In [1]:
import numpy as np
import os,sys,glob,json,pickle
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rc

import healpy as hp

import astropy.constants as const
import astropy.units as u
from astropy.cosmology import WMAP9 as cosmo
from astropy.coordinates import SkyCoord

import hasasia.sensitivity as hassen
import hasasia.sim as hassim
import hasasia.skymap as hassky

from enterprise.pulsar import Pulsar as ePulsar

In [2]:
rc('text',usetex=False)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':14})#,'weight':'bold'})

In [3]:
current_path = os.getcwd()
splt_path = current_path.split("/")
top_path_idx = splt_path.index('nanograv')
top_dir = "/".join(splt_path[0:top_path_idx+1])

pta_sim_dir = top_dir + '/pta_sim/pta_sim/'
sys.path.insert(0,pta_sim_dir)
import noise

In [4]:
psrs_mdc2_dir = top_dir + '/NANOGrav/MDC2/mdc2/group1/dataset_1a/'
noise_mdc2_file =  top_dir + '/NANOGrav/MDC2/mdc2/group1/group1_psr_noise.json'

pars_mdc2 = sorted(glob.glob(psrs_mdc2_dir+'*.par'))
tims_mdc2 = sorted(glob.glob(psrs_mdc2_dir+'*.tim'))
noise_mdc2 = noise.load_noise_files(noisepath=noise_mdc2_file)
print(len(pars_mdc2))

33


In [5]:
psrs_11yr_dir = top_dir + '/NANOGrav/BackgroundInjections/nano11/partim_new/'
noise_11yr_dir = top_dir + '/NANOGrav/BackgroundInjections/nano11/noisefiles_new/'

pars_11yr = sorted(glob.glob(psrs_11yr_dir+'*.par'))
tims_11yr = sorted(glob.glob(psrs_11yr_dir+'*.tim'))
noise_11yr = sorted(glob.glob(noise_11yr_dir+'*.txt'))
print(len(pars_11yr))

45


In [6]:
psrlist_11yr_file = top_dir + '/NANOGrav/BackgroundInjections/nano11/psrlist_Tg3yr.txt'
encoding = 'utf-8'
psr_list_bytes = np.loadtxt(psrlist_11yr_file,dtype='S42')
psr_list = []
for psr in psr_list_bytes:
    psr_list.append(psr.decode(encoding))
print(len(psr_list))

34


In [7]:
psrs_wn_only_dir = top_dir + '/NANOGrav/BackgroundInjections/FakePTA/'
pars_wn_only = sorted(glob.glob(psrs_wn_only_dir+'*.par'))
tims_wn_only = sorted(glob.glob(psrs_wn_only_dir+'*.tim'))
print(len(pars_wn_only))

36


In [8]:
def get_psrname(file,name_sep='_'):
    return file.split('/')[-1].split(name_sep)[0]

In [9]:
pars_11yr = [f for f in pars_11yr if get_psrname(f,name_sep='_') in psr_list]
tims_11yr = [f for f in tims_11yr if get_psrname(f,name_sep='_') in psr_list]
noise_11yr = [f for f in noise_11yr if get_psrname(f,name_sep='_') in psr_list]
len(pars_11yr), len(tims_11yr), len(noise_11yr)

(34, 34, 34)

In [10]:
pars_mdc2 = [f for f in pars_mdc2 if get_psrname(f,name_sep='.') in psr_list]
tims_mdc2 = [f for f in tims_mdc2 if get_psrname(f,name_sep='.') in psr_list]
noise_mdc2_11yr_psrs = {}
[noise_mdc2_11yr_psrs.update({f: noise_mdc2[f]}) for f in noise_mdc2.keys() if get_psrname(f,name_sep='_') in psr_list]
len(pars_mdc2), len(tims_mdc2), len(noise_mdc2)

(17, 17, 33)

In [11]:
#Get the pickled pulsars
injection_psr_file = top_dir + '/NANOGrav/BackgroundInjections' + \
    '/SimInject5/SimRuns/simGWB_1/enterprise_pickled_psrs.pickle'
with open(injection_psr_file, "rb") as f:
        ePsrs_injection = pickle.load(f)

In [13]:
#Get the pickled pulsars
injection_psr_file_cut = top_dir + '/NANOGrav/BackgroundInjections' + \
    '/SimInject5/SimRuns/simGWB_2/enterprise_pickled_psrs.pickle'
with open(injection_psr_file_cut, "rb") as f:
        ePsrs_injection_cut = pickle.load(f)

In [14]:
def get_epsrs_from_par_tim(pars,tims):
    ePsrs = []
    for par,tim in zip(pars,tims):
        ePsr = ePulsar(par, tim,  ephem='DE436')
        ePsrs.append(ePsr)
        print('\rPSR {0} complete'.format(ePsr.name),end='',flush=True)
    return ePsrs

In [15]:
ePsrs_11yr = get_epsrs_from_par_tim(pars_11yr,tims_11yr)

PSR B1953+29 complete

PSR J0030+0451 complete

PSR J0613-0200 complete

PSR J1600-3053 complete

PSR J1713+0747 complete

...

PSR J2010-1323 complete

PSR J2017+0603 complete

PSR J2145-0750 complete

PSR J2214+3000 complete

PSR J2317+1439 complete

In [16]:
ePsrs_mdc2 = get_epsrs_from_par_tim(pars_mdc2,tims_mdc2)

PSR J1600-3053 complete

PSR J1918-0642 complete

PSR J1944+0907 complete

PSR J2317+1439 complete

In [25]:
Tspan_11yr = hassen.get_Tspan(ePsrs_11yr)
Tspan_mdc2 = hassen.get_Tspan(ePsrs_mdc2)
Tspan_injection = hassen.get_Tspan(ePsrs_injection)
Tspan_injection_cut = hassen.get_Tspan(ePsrs_injection_cut)
print(Tspan_11yr/u.yr.to('s'),Tspan_mdc2/u.yr.to('s'),Tspan_injection/u.yr.to('s'),Tspan_injection_cut/u.yr.to('s'))

11.421371253218902 14.948695952444169 14.948697004605178 14.948696228263284


In [18]:
#Get Noise Values
noiseparams_mdc2 = noise.handle_noise_parameters(noise_mdc2_11yr_psrs)

In [19]:
noiseparams_injection = noise.handle_noise_parameters(noise_mdc2)

In [21]:
noiseparams_11yr = {}
for nf in noise_11yr:
    noiseparams_11yr.update(noise.get_noise_from_file(nf))

In [26]:
fyr = 1/(365.25*24*3600)
freqs_11yr = np.logspace(np.log10(1/(5*Tspan_11yr)),np.log10(2e-7),600)
freqs_mdc2 = np.logspace(np.log10(1/(5*Tspan_mdc2)),np.log10(2e-7),600)
freqs_injection = np.logspace(np.log10(1/(5*Tspan_injection)),np.log10(2e-7),600)
freqs_injection = np.logspace(np.log10(1/(5*Tspan_injection_cut)),np.log10(2e-7),600)

In [27]:
def make_corr(psr,noisedict=None):
    N = psr.toaerrs.size
    corr = np.zeros((N,N))
    _, _, U = hassen.quantize_fast(psr.toas,psr.toaerrs,dt=1)
    u = np.matmul(U,U.T)
    sigma_sqr = np.zeros(N)
    ecorrs = np.zeros(N)
    if noisedict != None:
        keys = [ky for ky in noisedict.keys() if psr.name in ky]
        if 'f' in psr.flags.keys():
            backends = np.unique(psr.flags['f'])
            for be in backends:
                mask = np.where(psr.flags['f']==be)
                key_ec = '{0}_{1}_log10_{2}'.format(psr.name,be,'ecorr')
                ecorrs[mask] = np.ones_like(mask) * (10**noisedict[key_ec])**2
                key_ef = '{0}_{1}_{2}'.format(psr.name,be,'efac')
                key_eq = '{0}_{1}_log10_{2}'.format(psr.name,be,'equad')
                sigma_sqr[mask] = (noisedict[key_ef]**2 * (psr.toaerrs[mask]**2)
                                   + (10**noisedict[key_eq])**2)
        else:
            key_ef = '{0}_{1}'.format(psr.name,'efac')
            key_eq = '{0}_log10_{1}'.format(psr.name,'equad')
            sigma_sqr = (noisedict[key_ef]**2 * (psr.toaerrs**2)
                               + (10**noisedict[key_eq])**2)
    
    corr = np.diag(sigma_sqr) + np.diag(u*ecorrs**2)
    return corr

In [28]:
def get_hasasia_psr_from_epsrs(ePsrs,freqs,noisedict=None,thin=0,rn_psrs=None):
    psrs = []
    for ePsr in ePsrs:
        plaw = hassen.red_noise_powerlaw(A=9e-16, gamma=13/3., freqs=freqs)
        if rn_psrs != None:
            if ePsr.name in rn_psrs.keys():
                Amp, gam = rn_psrs[ePsr.name]
                plaw += hassen.red_noise_powerlaw(A=Amp, gamma=gam, freqs=freqs)
                
        if thin == 0:
            corr = make_corr(ePsr,noisedict=noisedict)
            corr += hassen.corr_from_psd(freqs=freqs, psd=plaw,
                                       toas=ePsr.toas)
            psr = hassen.Pulsar(toas=ePsr.toas,
                              toaerrs=ePsr.toaerrs,
                              phi=ePsr.phi,theta=ePsr.theta,
                              N=corr, designmatrix=ePsr.Mmat)
        else:
            corr = make_corr(ePsr,noisedict=noisedict)[::thin,::thin]
            corr += hassen.corr_from_psd(freqs=freqs, psd=plaw,
                                       toas=ePsr.toas[::thin])
            psr = hassen.Pulsar(toas=ePsr.toas[::thin],
                              toaerrs=ePsr.toaerrs[::thin],
                              phi=ePsr.phi,theta=ePsr.theta,
                              N=corr, designmatrix=ePsr.Mmat[::thin,:])
            
        psr.name = ePsr.name
        psrs.append(psr)
        del ePsr
        print('\rPSR {0} complete'.format(psr.name),end='',flush=True)
    return psrs

psrs_11yr = get_hasasia_psr_from_epsrs(ePsrs_11yr,freqs_11yr,\
                                       noisedict=noiseparams_11yr,thin=10)

In [29]:
psrs_mdc2 = get_hasasia_psr_from_epsrs(ePsrs_mdc2,freqs_mdc2,\
                                          noisedict=noiseparams_mdc2)

PSR J2317+1439 complete

In [30]:
psrs_injection = get_hasasia_psr_from_epsrs(ePsrs_injection,freqs_injection,\
                                          noisedict=noiseparams_injection)

PSR J2317+1439 complete

In [ ]:
def get_hasasia_spec_from_psrs(psrs,freqs):
    specs = []
    for p in psrs:
        sp = hassen.Spectrum(p, freqs=freqs)
        _ = sp.NcalInv
        specs.append(sp)
        print('\rPSR {0} complete'.format(p.name),end='',flush=True)
    return specs

In [ ]:
spec_11yr = get_hasasia_spec_from_psrs(psrs_11yr,freqs_11yr)

In [ ]:
spec_mdc2 = get_hasasia_spec_from_psrs(psrs_mdc2,freqs_mdc2)

In [ ]:
spec_injection = get_hasasia_spec_from_psrs(psrs_injection,freqs_injection)

In [ ]:
names = ['J1713+0747','B1937+21','J1909-3744','J1024-0719']
for sp,p in zip(spec_11yr,psrs_11yr):
    if p.name in names:
        plt.loglog(sp.freqs,sp.h_c,lw=2,label=p.name)
    else:
        plt.loglog(sp.freqs,sp.h_c, color='k',lw=0.2)

plt.legend()
plt.show()
plt.close()

In [ ]:
names = ['J1713+0747','B1937+21','J1909-3744','J1024-0719']
for sp,p in zip(spec_mdc2,psrs_mdc2):
    if p.name in names:
        plt.loglog(sp.freqs,sp.h_c,lw=2,label=p.name)
    else:
        plt.loglog(sp.freqs,sp.h_c, color='k',lw=0.2)

plt.legend()
plt.show()
plt.close()

In [ ]:
names = ['J1713+0747','B1937+21','J1909-3744','J1024-0719']
for sp,p in zip(spec_injection,psrs_injection):
    if p.name in names:
        plt.loglog(sp.freqs,sp.h_c,lw=2,label=p.name)
    else:
        plt.loglog(sp.freqs,sp.h_c, color='k',lw=0.2)

plt.legend()
plt.show()
plt.close()

In [ ]:
sens_curve_11yr = hassen.GWBSensitivityCurve(spec_11yr)

In [ ]:
sens_curve_mdc2 = hassen.GWBSensitivityCurve(spec_mdc2)

In [ ]:
sens_curve_injection = hassen.GWBSensitivityCurve(spec_injection)

In [ ]:
plt.loglog(sens_curve_11yr.freqs,sens_curve_11yr.h_c)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Characteristic Strain, $h_c$')
plt.title('NANOGrav 11-year Data Set Sensitivity Curve')
plt.grid(which='both')
# plt.ylim(1e-15,9e-12)
plt.show()

In [ ]:
plt.loglog(sens_curve_mdc2.freqs,sens_curve_mdc2.h_c)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Characteristic Strain, $h_c$')
plt.title('MDC2 Sensitivity Curve')
plt.grid(which='both')
# plt.ylim(1e-15,9e-12)
plt.show()

In [ ]:
plt.loglog(sens_curve_injection.freqs,sens_curve_injection.h_c)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Characteristic Strain, $h_c$')
plt.title('Injection Sensitivity Curve')
plt.grid(which='both')
# plt.ylim(1e-15,9e-12)
plt.show()

In [ ]:
#First for alpha=-2/3 (the default value).
SNR=1
hgw_11yr=hassen.Agwb_from_Seff_plaw(sens_curve_11yr.freqs,
                             Tspan=Tspan_11yr,
                             SNR=SNR,
                             S_eff=sens_curve_11yr.S_eff)
plaw_h_11yr = hgw_11yr*(sens_curve_11yr.freqs/fyr)**(-2/3)

#And for an array of alpha values.
alpha = np.linspace(-7/4,5/4,30)
h_11yr=hassen.Agwb_from_Seff_plaw(freqs=sens_curve_11yr.freqs,Tspan=Tspan_11yr,SNR=SNR,
                           S_eff=sens_curve_11yr.S_eff,alpha=alpha)

plaw_11yr = np.dot((sens_curve_11yr.freqs[:,np.newaxis]/fyr)**alpha,
              h_11yr[:,np.newaxis]*np.eye(30))

In [ ]:
#First for alpha=-2/3 (the default value).
SNR=1
hgw_mdc2 =hassen.Agwb_from_Seff_plaw(sens_curve_mdc2.freqs,
                             Tspan=Tspan_mdc2,
                             SNR=SNR,
                             S_eff=sens_curve_mdc2.S_eff)
plaw_h_mdc2 = hgw_mdc2*(sens_curve_mdc2.freqs/fyr)**(-2/3)

#And for an array of alpha values.
alpha = np.linspace(-7/4,5/4,30)
h_mdc2=hassen.Agwb_from_Seff_plaw(freqs=sens_curve_mdc2.freqs,Tspan=Tspan_mdc2,SNR=SNR,
                           S_eff=sens_curve_mdc2.S_eff,alpha=alpha)

plaw_mdc2 = np.dot((sens_curve_mdc2.freqs[:,np.newaxis]/fyr)**alpha,
              h_mdc2[:,np.newaxis]*np.eye(30))

In [ ]:
#First for alpha=-2/3 (the default value).
SNR=1
hgw_injection =hassen.Agwb_from_Seff_plaw(sens_curve_injection.freqs,
                             Tspan=Tspan_injection,
                             SNR=SNR,
                             S_eff=sens_curve_injection.S_eff)
plaw_h_injection = hgw_injection*(sens_curve_injection.freqs/fyr)**(-2/3)

#And for an array of alpha values.
alpha = np.linspace(-7/4,5/4,30)
h_injection=hassen.Agwb_from_Seff_plaw(freqs=sens_curve_injection.freqs,Tspan=Tspan_injection,SNR=SNR,
                           S_eff=sens_curve_injection.S_eff,alpha=alpha)

plaw_injection = np.dot((sens_curve_injection.freqs[:,np.newaxis]/fyr)**alpha,
              h_injection[:,np.newaxis]*np.eye(30))

In [ ]:
for ii in range(len(h_11yr)):
    plt.loglog(sens_curve_11yr.freqs,plaw_11yr[:,ii],
               color='gray',lw=0.5)
plt.loglog(sens_curve_11yr.freqs,plaw_h_11yr,color='C1',lw=2,
           label='SNR={0}, '.format(SNR)+r'$\alpha=-2/3$')
plt.loglog(sens_curve_11yr.freqs,sens_curve_11yr.h_c, label='NG 11yr Sensitivity')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Characteristic Strain, $h_c$')
plt.axvline(fyr,linestyle=':')
plt.rc('text', usetex=True)
plt.title('NANOGrav 11-year Data Set\nPower-Law Interated Sensitivity Curve')
plt.ylim(hgw_11yr*0.75,2e-11)
plt.text(x=4e-8,y=3e-15,
         s=r'$A_{\rm GWB}$='+'{0:1.2e}'.format(hgw_11yr),
         bbox=dict(facecolor='white', alpha=0.6))
plt.legend(loc='upper left')
plt.show()

In [ ]:
for ii in range(len(h_mdc2)):
    plt.loglog(sens_curve_mdc2.freqs,plaw_mdc2[:,ii],
               color='gray',lw=0.5)
plt.loglog(sens_curve_mdc2.freqs,plaw_h_mdc2,color='C1',lw=2,
           label='SNR={0}, '.format(SNR)+r'$\alpha=-2/3$')
plt.loglog(sens_curve_mdc2.freqs,sens_curve_mdc2.h_c, label='MDC2 Sensitivity')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Characteristic Strain, $h_c$')
plt.axvline(fyr,linestyle=':')
plt.rc('text', usetex=True)
plt.title('MDC2 \nPower-Law Interated Sensitivity Curve')
plt.ylim(hgw_mdc2*0.75,2e-11)
plt.text(x=4e-8,y=3e-15,
         s=r'$A_{\rm GWB}$='+'{0:1.2e}'.format(hgw_mdc2),
         bbox=dict(facecolor='white', alpha=0.6))
plt.legend(loc='upper left')
plt.show()

In [ ]:
for ii in range(len(h_injection)):
    plt.loglog(sens_curve_injection.freqs,plaw_injection[:,ii],
               color='gray',lw=0.5)
plt.loglog(sens_curve_injection.freqs,plaw_h_injection,color='C1',lw=2,
           label='SNR={0}, '.format(SNR)+r'$\alpha=-2/3$')
plt.loglog(sens_curve_injection.freqs,sens_curve_injection.h_c, label='Injection Sensitivity')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Characteristic Strain, $h_c$')
plt.axvline(fyr,linestyle=':')
plt.rc('text', usetex=True)
plt.title('Injection \nPower-Law Interated Sensitivity Curve')
plt.ylim(hgw_injection*0.75,2e-11)
plt.text(x=4e-8,y=3e-15,
         s=r'$A_{\rm GWB}$='+'{0:1.2e}'.format(hgw_injection),
         bbox=dict(facecolor='white', alpha=0.6))
plt.legend(loc='upper left')
plt.show()